In [60]:
import numpy as np
from bilby.core.result import read_in_result
from bilby.hyper.model import Model
from gwpopulation.models.redshift import PowerLawRedshift
from tqdm import tqdm

from gwpop.mass_models import matter_matters_pairing
from gwpop.spin_models import (
    iid_spin_magnitude_beta,
    iid_spin_orientation_gaussian_isotropic,
)
from pipe.gwpopulation_pipe_pdb import draw_true_values

## 1. CONFIGURATION

In [61]:
RESULT_FILE = "data/baseline5_widesigmachi2_mass_NotchFilterBinnedPairingMassDistribution_redshift_powerlaw_mag_iid_spin_magnitude_gaussian_tilt_iid_spin_orientation_result.hdf5"
N_SAMPLES = int(1e6)
CHUNK_SIZE = int(1e3)
Z_MAX = 2.3


## 2. LOAD MAP PARAMETERS FROM BILBY RESULT FILE

In [62]:
# Load "Broken Power Law + 2 Peaks model" result
result = read_in_result(RESULT_FILE)
posterior = result.posterior.copy()

In [63]:
# Find the sample with maximum likelihood (same as max posterior for uniform priors)
max_likelihood_sample = posterior.loc[np.argmax(posterior.log_likelihood)]

In [64]:
# Set minimum and maximum allowed masses for the model (can be tuned)
max_likelihood_sample["absolute_mmin"] = 0.5
max_likelihood_sample["absolute_mmax"] = 350

In [65]:
print("Max parameters:")
print(max_likelihood_sample)

Max parameters:
A                  0.091462
A2                 0.828165
BHmax            152.055979
BHmin              7.763955
NSmax              4.094744
                    ...    
var_99             0.001657
variance           0.984675
xi_spin            0.712869
absolute_mmin      0.500000
absolute_mmax    350.000000
Name: 2411, Length: 334, dtype: float64


In [66]:
# Use absolute
exclude_abs = {"alpha_1", "alpha_2", "log_prior"}
for key in max_likelihood_sample.index:
    if key not in exclude_abs:
        max_likelihood_sample[key] = np.abs(max_likelihood_sample[key])

max_likelihood_sample

A                  0.091462
A2                 0.828165
BHmax            152.055979
BHmin              7.763955
NSmax              4.094744
                    ...    
var_99             0.001657
variance           0.984675
xi_spin            0.712869
absolute_mmin      0.500000
absolute_mmax    350.000000
Name: 2411, Length: 334, dtype: float64


## 3. CONSTRUCT POPULATION MODEL WITH MAP PARAMETERS

In [67]:
# Compose the model: mass, spin orientation, spin magnitude, redshift
model = Model(
    [
        matter_matters_pairing,
        iid_spin_orientation_gaussian_isotropic,
        iid_spin_magnitude_beta,
        PowerLawRedshift(z_max=Z_MAX),
    ]
)

model

In [68]:
# Set the parameters from MAP sample
model.parameters.update(max_likelihood_sample)

## 4. DRAW SYNTHETIC POPULATION SAMPLES (IN CHUNKS)

In [69]:
n_chunks = int(np.ceil(N_SAMPLES / CHUNK_SIZE))
all_events = []

for i in tqdm(range(n_chunks), desc="Simulating events"):
    current_chunk_size = min(CHUNK_SIZE, N_SAMPLES - i * CHUNK_SIZE)
    events = draw_true_values(
        model=model,
        vt_model=None,
        n_samples=current_chunk_size,
        parameters=None,
    )
    # all_events.append(events)

Simulating events:   0%|          | 0/1000 [00:00<?, ?it/s]/Users/rk281962/Projects/CBC/GWTC-4-distribution/utils/cupy_utils.py:198: RuntimeWarning: invalid value encountered in log
  ln_beta = (alpha - 1) * xp.log(xx) + (beta - 1) * xp.log(scale - xx)


model.prob(data).shape: (10000000,)


02:16 bilby INFO    : Sampling efficiency low. Total samples so far: 174653
/Users/rk281962/Projects/CBC/GWTC-4-distribution/utils/cupy_utils.py:198: RuntimeWarning: invalid value encountered in log
  ln_beta = (alpha - 1) * xp.log(xx) + (beta - 1) * xp.log(scale - xx)
Simulating events:   0%|          | 0/1000 [00:06<?, ?it/s]

model.prob(data).shape: (2,)


ValueError: operands could not be broadcast together with shapes (2,) (10000000,) 


## 5. If we need to replace alpha_1 and alpha_chi  values

In [70]:
max_likelihood_sample["alpha_chi"] = 1
max_likelihood_sample["alpha_1"] = 1

In [71]:
N_SAMPLES = int(10000)

# Compose the model: mass, spin orientation, spin magnitude, redshift
model = Model(
    [
        matter_matters_pairing,
        iid_spin_orientation_gaussian_isotropic,
        iid_spin_magnitude_beta,
        PowerLawRedshift(z_max=Z_MAX),
    ]
)

# Set the parameters from MAP sample
model.parameters.update(max_likelihood_sample)

n_chunks = int(np.ceil(N_SAMPLES / CHUNK_SIZE))
all_events = []

for i in tqdm(range(n_chunks), desc="Simulating events"):
    current_chunk_size = min(CHUNK_SIZE, N_SAMPLES - i * CHUNK_SIZE)
    events = draw_true_values(
        model=model,
        vt_model=None,
        n_samples=current_chunk_size,
        parameters=None,
    )
    # all_events.append(events)

Simulating events:   0%|          | 0/10 [00:00<?, ?it/s]/Users/rk281962/Projects/CBC/GWTC-4-distribution/gwpop/mass_models.py:125: RuntimeWarning: overflow encountered in power
  highpass_lower = 1 + (NSmin / mass) ** n0
/Users/rk281962/Projects/CBC/GWTC-4-distribution/gwpop/mass_models.py:126: RuntimeWarning: overflow encountered in power
  notch_lower = 1.0 - A / ((1 + (NSmax / mass) ** n1) * (1 + (mass / BHmin) ** n2))
/Users/rk281962/Projects/CBC/GWTC-4-distribution/utils/cupy_utils.py:198: RuntimeWarning: invalid value encountered in log
  ln_beta = (alpha - 1) * xp.log(xx) + (beta - 1) * xp.log(scale - xx)


model.prob(data).shape: (10000000,)


Simulating events:  10%|█         | 1/10 [00:05<00:53,  5.96s/it]

model.prob(data).shape: (10000000,)


Simulating events:  20%|██        | 2/10 [00:07<00:26,  3.26s/it]

...

Simulating events:  80%|████████  | 8/10 [00:15<00:02,  1.45s/it]

model.prob(data).shape: (10000000,)


Simulating events:  90%|█████████ | 9/10 [00:16<00:01,  1.39s/it]

model.prob(data).shape: (10000000,)


Simulating events: 100%|██████████| 10/10 [00:17<00:00,  1.79s/it]
